In [1]:
import pandas as pd
import numpy as np

In [2]:
def Jpmaker(i):
    if len(str(i)) == 2:
        jp = '0' + str(i)
    elif len(str(i)) == 1:
        jp = '00' + str(i)
    else:
        jp = i
    return jp

# Cargamos la Info

In [3]:
df = pd.read_excel('BaseEjemplo.xlsx' , sheet_name="Base") #Base de datos
codigos = pd.read_excel('BaseEjemplo.xlsx' , sheet_name='codigos') #codigos de centros
codigos.set_index('Indice',inplace = True)
Jp = pd.read_excel('BaseJP.xlsx' , sheet_name='ss') # Escoger si es ing o ss. Para ing escribir ing, para ss escribir ss
Jp.set_index('EfDesc' , inplace = True)
total = df.shape[0]
nv = df['NVNumero'].unique()
date = "31-10-9999" #Fecha

# PreProcessing

In [4]:
provventa = []
for i in range(total):
    if df.iloc[i]['COLUMNA2'] == 'NORMAL':
        provventa.append("AA")
    else:
        provventa.append("BB")
df['provVenta'] = provventa
clasf = []
for i in range(total):
    if df.iloc[i]['DesProd'] == 'AREA2':
        clasf.append(5)
    else:
        clasf.append(4)
centronum = []
for i in range(total):
    centronum.append(codigos.loc[df.iloc[i]['CodProd']]['CC'] + str(df.iloc[i]['CotNum']))
df['provVenta'] = provventa
df['clasf'] = clasf
df['centronum'] = centronum
df.set_index('NVNumero', inplace = True)

In [5]:
comprobante = []
for c,i in enumerate(nv):
    data = df.loc[i]
    ninput = df.loc[i].shape[0] # Numero de Inputs por Nv
    try: # para casos que no son un registro
        for j in range(ninput):
            current = data.iloc[j]
            d = {
                'index' : i,
                'cuenta contable': codigos.loc[current['CodProd']]['Cuenta'] ,
                'subtotal' : "",
                'castigo' : current['nvPrecio80'],
                'PROVVENTA' : "PROV DE VENTA " + data['provVenta'].unique()[0] + str(data["COLUMNA3"].unique()[0]),
                'centro': current['centronum'],
                'rut': current['CodAux'],
                'ep': "",
                'num' : "",
                'fecha': date,
                'fecha2' : date,
                'ep2': "",
                'num2' : "",
                'indicecpb': c+1,
                'JP': Jpmaker(Jp.loc[current['COLUMNA4']]['EfCodi']),
                'subtotal2' : "",

            }
            comprobante.append(d)
        ######## SEPARACION ING / CONSULTORIA #####
        vals = data['clasf'].unique() # Obtener clasificación
        data2 = data.copy()
        data2.set_index('clasf' , inplace = True)
        sverif = data2.shape[0] #verificamos la shape del array
        for k in vals:
            if k == 4: ## si hay un registro en ingeniería 
                if sverif > 2: 
                    d = {
                        'index' : i,
                        'cuenta contable': "1-1-03-02-01" ,
                        'subtotal' : data2.loc[k]['nvPrecio80'].sum(),
                        'castigo': "",
                        'PROVVENTA' : "PROV DE VENTA " + data['provVenta'].unique()[0] + str(data["COLUMNA3"].unique()[0]),
                        'centro': data2.loc[k]['centronum'].values[-1],
                        'rut': current['CodAux'],
                        'ep' : current['provVenta'],
                        'num' : current['COLUMNA3'],
                        'fecha': date,
                        'fecha2' : date,
                        'ep2' : current['provVenta'],
                        'num2' : current['COLUMNA3'],
                        'indicecpb': c+1,
                        'JP': Jpmaker(Jp.loc[current['COLUMNA4']]['EfCodi']),
                        'subtotal2' : data2.loc[k]['nvPrecio80'].sum()

                    }
                    comprobante.append(d)
                elif sverif == 2 and len(data2.index.unique()) == 1:
                    d = {
                        'index' : i,
                        'cuenta contable': "1-1-03-02-01" ,
                        'subtotal' : data2.loc[k]['nvPrecio80'].sum(),
                        'castigo': "",
                        'PROVVENTA' : "PROV DE VENTA " + data['provVenta'].unique()[0] + str(data["COLUMNA3"].unique()[0]),
                        'centro': data2.loc[k]['centronum'].values[-1],
                        'rut': current['CodAux'],
                        'ep' : current['provVenta'],
                        'num' : current['COLUMNA3'],
                        'fecha': date,
                        'fecha2' : date,
                        'ep2' : current['provVenta'],
                        'num2' : current['COLUMNA3'],
                        'indicecpb': c+1,
                        'JP': Jpmaker(Jp.loc[current['COLUMNA4']]['EfCodi']),
                        'subtotal2' : data2.loc[k]['nvPrecio80'].sum()

                    }
                    comprobante.append(d)
                else:
                    d = {
                        'index' : i,
                        'cuenta contable': "1-1-03-02-01" ,
                        'subtotal' : data2.loc[k]['nvPrecio80'].sum(),
                        'castigo': "",
                        'PROVVENTA' : "PROV DE VENTA " + data['provVenta'].unique()[0] + str(data["COLUMNA3"].unique()[0]),
                        'centro': data2.loc[k]['centronum'],
                        'rut': current['CodAux'],
                        'ep' : current['provVenta'],
                        'num' : current['COLUMNA3'],
                        'fecha': date,
                        'fecha2' : date,
                        'ep2' : current['provVenta'],
                        'num2' : current['COLUMNA3'],
                        'indicecpb': c+1,
                        'JP': Jpmaker(Jp.loc[current['COLUMNA4']]['EfCodi']),
                        'subtotal2' : data2.loc[k]['nvPrecio80'].sum()

                        }
                    comprobante.append(d)     
            else:
                d = {
                    'index' : i,
                    'cuenta contable': "1-1-03-02-01" ,
                    'subtotal' : data2.loc[k]['nvPrecio80'].sum(),
                    'castigo': "",
                    'PROVVENTA' : "PROV DE VENTA " + data['provVenta'].unique()[0] + str(data["COLUMNA3"].unique()[0]),
                    'centro': data2.loc[k]['centronum'],
                    'rut': current['CodAux'],
                    'ep' : current['provVenta'],
                    'num' : current['COLUMNA3'],
                    'fecha': date,
                    'fecha2' : date,
                    'ep2' : current['provVenta'],
                    'num2' : current['COLUMNA3'],
                    'indicecpb': c+1,
                    'JP': Jpmaker(Jp.loc[current['COLUMNA4']]['EfCodi']),
                    'subtotal2' : data2.loc[k]['nvPrecio80'].sum()

                }
                comprobante.append(d)  
    except:
        d = {
            'index' : i,
            'cuenta contable': codigos.loc[data['CodProd']]['Cuenta'] ,
            'subtotal' : "",
            'castigo': data['nvPrecio80'],
            'PROVVENTA' : "PROV DE VENTA " + data['provVenta'] + str(data["COLUMNA3"]),
            'centro': data['centronum'],
            'rut': data['CodAux'],
            'ep' : "",
            'num': "",
            'fecha': date,
            'fecha2' : date,
            'ep2' : "",
            'num2': "",
            'indicecpb': c+1,
            'JP': Jpmaker(Jp.loc[data['COLUMNA4']]['EfCodi']),
            'subtotal2' : ""
        }
        comprobante.append(d)
        d = {
            'index' : i,
            'cuenta contable': "1-1-03-02-01" ,
            'subtotal' : df.loc[i]['nvPrecio80'],
            'castigo': "",
            'PROVVENTA' : "PROV DE VENTA " + data['provVenta'] + str(data["COLUMNA3"]),
            'centro': data['centronum'],
            'rut': data['CodAux'],
            'ep': data['provVenta'],
            'num' : data['COLUMNA3'],
            'fecha': date,
            'fecha2' : date,
            'ep2': data['provVenta'],
            'num2' : data['COLUMNA3'],
            'indicecpb': c+1,
            'JP': Jpmaker(Jp.loc[data['COLUMNA4']]['EfCodi']),
            'subtotal2' : df.loc[i]['nvPrecio80']
        }
        comprobante.append(d)

# Exportar Resultado

In [6]:
ans = pd.DataFrame(comprobante)

In [7]:
ans.to_excel('comprobante.xlsx')